Imports

In [159]:
import pandas as pd
import numpy as np
import requests
import plotly.express as px
from sklearn import preprocessing
from sklearn.impute import KNNImputer

Reviewing a sample row from each file

In [160]:
acquired = pd.read_csv("Data/Acquired Tech Companies.csv")
acquired.iloc[0]

Company                                                     Day Software
CrunchBase Profile     http://www.crunchbase.com/organization/day-sof...
Image                  http://a5.images.crunchbase.com/image/upload/c...
Tagline                Day Software develops web applications that al...
Year Founded                                                         NaN
Market Categories                                               Software
Address (HQ)           Barfüsserplatz 6, Basel, Basel-Stadt, Switzerland
City (HQ)                                                          Basel
State / Region (HQ)                                          Basel-Stadt
Country (HQ)                                                 Switzerland
Description            Day was founded in Basel, Switzerland, in 1993...
Homepage                                              http://www.day.com
Twitter                                                              NaN
Acquired by                                        

In [161]:
acquiring = pd.read_csv("Data/Acquiring Tech Companies.csv")
acquiring.iloc[0]

Acquiring Company                                                                        Adobe
CrunchBase Profile                               www.crunchbase.com/organization/adobe-systems
Image                                        http://a2.images.crunchbase.com/image/upload/c...
Tagline                                      Adobe is an American multinational computer so...
Market Categories                            Photo Editing, Design, Creative, Software, Ima...
Year Founded                                                                              1982
IPO                                                                                       1986
Founders                                                         John Warnock, Charles Geschke
Number of Employees                                                                     11,144
Number of Employees (year of last update)                                               2012.0
Total Funding ($)                                 

In [162]:
acquisitions = pd.read_csv("Data/Acquisitions.csv")
acquisitions.iloc[0]

Acquisitions ID                                      EMC acquired Data Domain in 2009
Acquired Company                                                          Data Domain
Acquiring Company                                                                 EMC
Year of acquisition announcement                                                 2009
Deal announced on                                                           8/07/2009
Price                                                                  $2,100,000,000
Status                                                                    Undisclosed
Terms                                                                            Cash
Acquisition Profile                 http://www.crunchbase.com/acquisition/5dc676a1...
News                                                         EMC acquired Data Domain
News Link                           http://www.businesswire.com/news/home/20090708...
Name: 0, dtype: object

In [163]:
founders = pd.read_csv("Data/Founders and Board Members.csv")
founders.iloc[0]

Name                                                 Hans-Werner Hector
CrunchBase Profile      http://de.wikipedia.org/wiki/Hans-Werner_Hector
Role                                                            Founder
Companies                                                           SAP
Image                 http://images.forbes.com/media/lists/10/2006/4...
Name: 0, dtype: object

We will link between the files using these columns:
* Acquisitions ID to link the acquisitions
* 'Founders' and 'Name' to link the Founders

In [164]:
np.intersect1d(acquired.columns, acquisitions.columns).tolist()

['Acquisitions ID']

In [165]:
np.intersect1d(acquiring.columns, acquisitions.columns).tolist()

['Acquiring Company', 'Acquisitions ID']

In [166]:
def ValidateLink(url, timeout=15):
    session = requests.Session()
    # fake headers to make it seem like a real request
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "DNT": "1",
    }
    session.headers.update(headers)
    try:
        response = session.get(url, timeout=timeout, allow_redirects=True, stream=True)
        status_code = response.status_code
        response.close()
        if status_code < 400:
            return True
        else:
            return False
    except Exception as e:
        return False

In [167]:
def ValidateLinks(urls):
    results = []
    for url in urls:
        results.append(ValidateLink(url))
        if results[-1]:
            return results
    return results

In [168]:
def ValidateLinksDF(df):
    for col in df.columns:
        for val in df[col]:
            if type(val) == str and ("http" in val):
                print(col)
                results = ValidateLinks(df[col])
                if not pd.Series(results).any():
                    print(f'Column "{col}" had no valid links , or is using captcha.')
                    print("Try it yourself:")
                    print(df[col][0] + "\n")
                break

ValidateLinksDF(acquired)

* CrunchBase is using CAPTCHA , so we won't drop it now but we will process it later
* Image links are all corrupt so we will drop the column 

In [169]:
acquired = acquired.drop("Image", axis=1)

ValidateLinksDF(acquiring)

* drop Image also

In [170]:
acquiring = acquiring.drop("Image", axis=1)

ValidateLinksDF(acquisitions)

* acquisitions profile is also a crunchbase link

In [171]:
acquisitions = acquisitions.drop("Acquisition Profile", axis=1)

ValidateLinksDF(founders)

We don't need the exact address of the company, we already have the city , state and country

In [172]:
acquired = acquired.drop("Address (HQ)", axis=1)
acquiring = acquiring.drop("Address (HQ)", axis=1)

**Adding the target variable**

In [173]:
acquisitions["Price"] = [
    int(price.removeprefix("$").replace(",", "")) for price in acquisitions["Price"]
]

In [174]:
acquired["Price"] = None
acquired["Year of acquisition announcement"] = None

In [175]:
for i, company in enumerate(acquisitions["Acquired Company"]):
    acquired.loc[acquired["Company"] == company, "Price"] = acquisitions.iloc[i][
        "Price"
    ]
    acquired.loc[acquired["Company"] == company, "Year of acquisition announcement"] = (
        acquisitions.iloc[i]["Year of acquisition announcement"]
    )

In [176]:
fig = px.scatter(
    acquisitions,
    x="Year of acquisition announcement",
    y="Price",
    title="Acquisition Price by Year",
    width=600,
    height=400,
)
fig.show()

There was a wrongly entered value, so I looked at the link and corrected it

In [177]:
acquisitions.loc[
    acquisitions["Year of acquisition announcement"] == 2104,
    "Year of acquisition announcement",
] = 2014

Plotting again without the error, now we can see that the overall trend of prices tends to go up, that's why we added the 'Year of acquisitions announcement' column

In [178]:
fig = px.scatter(
    acquisitions,
    x="Year of acquisition announcement",
    y="Price",
    title="Acquisition Price by Year",
    width=700,
    height=400,
)
fig.show()

update the datatypes automatically

In [179]:
acquired = acquired.infer_objects()
acquisitions = acquisitions.infer_objects()

In [180]:
fig = px.scatter(
    acquired,
    x="Year Founded",
    y="Price",
    title="Acquisition Price by Year",
    width=600,
    height=400,
)
fig.show()

Another error found and corrected

In [181]:
acquired.loc[acquired["Year Founded"] == 1840, "Year Founded"] = 2006
acquired.loc[acquired["Year Founded"] == 1933, "Year Founded"] = 1989

In [182]:
fig = px.scatter(
    acquired,
    x="Year Founded",
    y="Price",
    title="Acquisition Price by Year",
    width=600,
    height=400,
)
fig.show()

In [183]:
acquired.iloc[12]["Tagline"]

'5min Media is a syndication platform for lifestyle, knowledge and instructional videos.'

In [184]:
for l in acquired.iloc[12]["Description"].split("."):
    print(l + "\n")

5min Media is the leading syndication platform for lifestyle, knowledge and instructional videos

 Reinventing the cable network online, 5min reaches engaged and targeted audiences of passionate consumers through its network of 100s of lifestyle and niche websites

 The 5min video library comprises more than 200,000 short-form videos from some of the world's largest media companies, as well as the most innovative independent producers

 Visit  for more information





* 'Tagline' contains a brief and precise description of the company , while the 'Description' is very long and doesn't provide any more important details, 
so we will drop the 'Description'

In [185]:
acquiring = acquiring.drop("Description", axis=1)
acquired = acquired.drop("Description", axis=1)

### There isn't any new useful information that we can get out of those , so we will drop them

* "CrunchBase Profile" and "API" columns are both on the crunchbase website , which uses captcha so we can't scrap it, and their API is paid , and the provided API key is invalid , so we can't use it

* "Homepage" column contains the link to the website of every company , and they aren't all the same so we can't apply a function or a program to extract certain information about them. To use the link , this would require us to go over into each of them one by one , which isn't  feasible


* "Twitter" column also can't be scraped according to their new policy , tried multiple APIs and libraries but none of them worked , even twitter's free tier API is useless
 

* "Acquisition ID" is just used to link between files , and we can do that with the company's name


In [186]:
acquired = acquired.drop(["CrunchBase Profile", "Homepage", "Twitter", "API"], axis=1)
acquiring = acquiring.drop(
    ["CrunchBase Profile", "Homepage", "Twitter", "Acquisitions ID", "API"], axis=1
)
founders = founders.drop("CrunchBase Profile", axis=1)

In [187]:
acquired["Age on acquisition"] = (
    acquired["Year of acquisition announcement"] - acquired["Year Founded"]
)

In [188]:
acquired = acquired.drop(["Year Founded", "Year of acquisition announcement"], axis=1)

All these columns are probably related to the target column , so we will keep them for now

Market categories contains multiple values , still not processed

In [189]:
acquired.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Company              310 non-null    object 
 1   Tagline              307 non-null    object 
 2   Market Categories    287 non-null    object 
 3   City (HQ)            275 non-null    object 
 4   State / Region (HQ)  273 non-null    object 
 5   Country (HQ)         276 non-null    object 
 6   Acquired by          309 non-null    object 
 7   Acquisitions ID      310 non-null    object 
 8   Price                310 non-null    int64  
 9   Age on acquisition   241 non-null    float64
dtypes: float64(1), int64(1), object(8)
memory usage: 24.3+ KB


Dropping 'year of last update' of the number of employees , because we don't need it directly and can't use it in any way to pridct the current number

In [190]:
acquiring = acquiring.drop("Number of Employees (year of last update)", axis=1)

There are multiple 'NOT YET' in the IPO column , and the earliest the number the better it is , so we won't replace them with zero ,we will replace them with 2025 or anything larger

In [191]:
acquiring["IPO"].value_counts()[:5]

IPO
1986       4
1978       4
1983       2
Not yet    2
1990       2
Name: count, dtype: int64

In [192]:
acquiring.loc[acquiring["IPO"] == "Not yet", "IPO"] = 2025  # 2025 is debatable

In [193]:
acquiring["Number of Employees"] = [
    int(n.replace(",", "")) if type(n) != float else n
    for n in acquiring["Number of Employees"]
]

Idea for acquiring companies: calculate the average price paid for all acquired companies

how to categorize multiple values in the same cell?

In [194]:
acquiring["Market Categories"][:5]

0    Photo Editing, Design, Creative, Software, Ima...
1    Groceries, Consumer Goods, Crowdsourcing, E-Co...
2    News, Advertising Platforms, Content Creators,...
3    Computers, Consumer Electronics, Hardware + So...
4                                               Mobile
Name: Market Categories, dtype: object

In [195]:
acquiring = acquiring.astype(
    {
        "IPO": "float",
    }
)

In [196]:
flattened = [x for item in acquiring["Board Members"].dropna() for x in item.split(",")]

In [197]:
pd.Series(flattened).nunique()

309

In [198]:
len(np.intersect1d(founders["Name"], flattened))

34

Some of the board members are in the founders df , so we won't drop them for now

In [199]:
acquiring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Acquiring Company       36 non-null     object 
 1   Tagline                 36 non-null     object 
 2   Market Categories       36 non-null     object 
 3   Year Founded            36 non-null     int64  
 4   IPO                     35 non-null     float64
 5   Founders                36 non-null     object 
 6   Number of Employees     35 non-null     float64
 7   Total Funding ($)       36 non-null     int64  
 8   Number of Acquisitions  36 non-null     int64  
 9   Board Members           34 non-null     object 
 10  City (HQ)               34 non-null     object 
 11  State / Region (HQ)     33 non-null     object 
 12  Country (HQ)            36 non-null     object 
 13  Acquired Companies      36 non-null     object 
dtypes: float64(2), int64(3), object(9)
memory us

In [200]:
founders["Companies"].value_counts()[:5]

Companies
Microsoft                 21
IBM                       20
Cisco Systems             17
Verizon Communications    15
Nokia                     15
Name: count, dtype: int64

In [201]:
founders["Role"].value_counts()

Role
Board of Directors                    222
Founder                                79
Advisory Board                         72
Board of Directors, Advisory Board      4
Board Observer                          3
Board of Directors, Founder             2
Name: count, dtype: int64

The image of the founder doesn't affect anything at all ... DROPPED

In [202]:
founders = founders.drop("Image", axis=1)

Ready

In [203]:
founders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382 entries, 0 to 381
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Name       382 non-null    object
 1   Role       382 non-null    object
 2   Companies  382 non-null    object
dtypes: object(3)
memory usage: 9.1+ KB


* The specific date which the deal was announced on doesn't matter , what matters is the year so the model can know that inflation affects the price
* The ID doesn't add any new info
* The News and News link don't add any info or details about the acquisition

In [204]:
acquisitions = acquisitions.drop(["Deal announced on", "News", "News Link"], axis=1)

In [205]:
acquisitions["Status"].value_counts()

Status
Undisclosed    310
Complete        16
Pending          9
Name: count, dtype: int64

In [206]:
acquisitions["Terms"].value_counts()

Terms
Undisclosed    148
Cash           128
Cash, Stock     36
Stock           24
Name: count, dtype: int64

In [207]:
acquisitions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 7 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Acquisitions ID                   336 non-null    object
 1   Acquired Company                  336 non-null    object
 2   Acquiring Company                 336 non-null    object
 3   Year of acquisition announcement  336 non-null    int64 
 4   Price                             336 non-null    int64 
 5   Status                            335 non-null    object
 6   Terms                             336 non-null    object
dtypes: int64(2), object(5)
memory usage: 18.5+ KB


In [208]:
df = acquired.copy()

In [209]:
renamed_columns = {}
for col in acquiring.columns:
    new_col = f"{col} (Acquiring)"
    renamed_columns[col] = new_col

acquiring = acquiring.rename(columns=renamed_columns)

for col in acquiring.columns:
    if col not in df.columns:
        df[col] = None

for i, row1 in df.iterrows():
    for j, row2 in acquiring.iterrows():
        if row1["Acquired by"] == row2["Acquiring Company (Acquiring)"]:
            for col in acquiring.columns:
                df.at[i, col] = row2[col]

In [210]:
df = df.drop("Acquired by", axis=1)  # delete a the duplicate column used for linking

In [211]:
renamed_columns = {}
for col in acquisitions.columns:
    new_col = f"{col} (Acquisitions)"
    renamed_columns[col] = new_col

acquisitions = acquisitions.rename(columns=renamed_columns)

for col in acquisitions.columns:
    if col not in df.columns:
        df[col] = None

for i, row1 in df.iterrows():
    for j, row2 in acquisitions.iterrows():
        if row1["Acquisitions ID"] == row2["Acquisitions ID (Acquisitions)"]:
            for col in acquisitions.columns:
                df.at[i, col] = row2[col]

Delete duplicate columns , and already used columns

In [212]:
df = df.drop(
    [
        "Acquisitions ID",
        "Acquiring Company (Acquisitions)",
        "Acquired Company (Acquisitions)",
        "Acquisitions ID (Acquisitions)",
        "Price (Acquisitions)",
        "Year of acquisition announcement (Acquisitions)",
    ],
    axis=1,
)

In [213]:
df.loc[0]

Company                                                                    Day Software
Tagline                               Day Software develops web applications that al...
Market Categories                                                              Software
City (HQ)                                                                         Basel
State / Region (HQ)                                                         Basel-Stadt
Country (HQ)                                                                Switzerland
Price                                                                         240000000
Age on acquisition                                                                  NaN
Acquiring Company (Acquiring)                                                     Adobe
Tagline (Acquiring)                   Adobe is an American multinational computer so...
Market Categories (Acquiring)         Photo Editing, Design, Creative, Software, Ima...
Year Founded (Acquiring)        

renamed_columns = {}
for col in founders.columns:
    new_col = f"{col} (Founders)"
    renamed_columns[col] = new_col

founders = founders.rename(columns=renamed_columns)

for col in founders.columns:
    if col not in df.columns:
        df[col] = None

for j, row1 in founders.iterrows():
    for i, row2 in df.iterrows():
        if row1["Name (Founders)"] in row2["Founders (Acquiring)"]:
            for col in founders.columns:
                df.at[i, col] = row2[col]

### Spliting each multi-valued category to an array of categories

In [214]:
def SplitMultiValuedColumn(df, label):
    df[label] = [
        [
            value.strip() if type(value) == str else value
            for value in str(values).split(",")
        ]
        for values in df[label]
    ]

In [215]:
def getUniqueLabels(df, label):
    uniqueLabels = []
    for labels in df[label].dropna():
        for label in labels:
            if [label] not in uniqueLabels:
                uniqueLabels.append(label)
    return np.ravel(uniqueLabels)

In [216]:
def encodeMultiValuedCategory(df, label: str, categories=[]):
    le = preprocessing.LabelEncoder()
    SplitMultiValuedColumn(df, label)
    if len(categories) == 0:
        categories = getUniqueLabels(df, label)
    le.fit(categories)
    df[label] = [le.transform(values) for values in df[label].dropna()]
    return le.classes_

In [217]:
def encodeCategory(df, label: str, categories=[]):
    nonNullIndex = df[label].notna()

    le = preprocessing.LabelEncoder()
    if len(categories) == 0:
        categories = df.loc[nonNullIndex, label]

    le.fit(categories)
    df.loc[nonNullIndex, label] = le.transform(df.loc[nonNullIndex, label])
    return le.classes_

In [218]:
def FindMultiValuedColumns(df):
    cols = []
    for col in df.columns:
        try:  # To skip numeric columns
            if (
                len(
                    [
                        value
                        for value in df[col].dropna().values
                        if len(value.split(",")) > 1
                    ]
                )
                > 1
            ):
                cols.append(col)
        except:
            pass
    return cols

In [219]:
FindMultiValuedColumns(df)

['Tagline',
 'Market Categories',
 'Tagline (Acquiring)',
 'Market Categories (Acquiring)',
 'Founders (Acquiring)',
 'Board Members (Acquiring)',
 'Acquired Companies (Acquiring)',
 'Terms (Acquisitions)']

In [220]:
encoded = np.intersect1d(df.columns, FindMultiValuedColumns(df))

In [221]:
for label in FindMultiValuedColumns(df.drop("Tagline", axis=1)):
    print(encodeMultiValuedCategory(df, label)[:5])

['3D' 'Advertising' 'Aerospace' 'Analytics' 'Android']
["' messages limited to 140 characters." '3G network'
 'AOL is a Brand company committed to innovating'
 'AT&T provides telecommunications services such as wireless communications'
 'Adobe is an American multinational computer software company that provides digital marketing and digital media solutions.']
['Advertising Platforms' 'All Markets' 'All Students' 'Auctions'
 'Big Data']
['Akio Morita' 'Alexander Graham Bell' 'Allen Blue' 'Andrew McCollum'
 'Andrew Viterbi']
['AJ Jaghori' 'Adam Grosser' 'Al Gore' 'Alex J. Mandl'
 'Alexander Gounares']
['2Web Technologies' '2dehands.be' '2ememain.be' '360Commerce'
 '3721 Internet Assistant']
['Cash' 'Stock' 'Undisclosed']


In [222]:
df["Terms (Acquisitions)"][:5]

0    [0]
1    [2]
2    [1]
3    [2]
4    [1]
Name: Terms (Acquisitions), dtype: object

In [223]:
for col in df.drop(encoded, axis=1).columns:
    encodeCategory(df, col)

In [224]:
df.loc[0]

Company                                                                              66
Tagline                               Day Software develops web applications that al...
Market Categories                                                                 [117]
City (HQ)                                                                             8
State / Region (HQ)                                                                   2
Country (HQ)                                                                         13
Price                                                                                81
Age on acquisition                                                                  NaN
Acquiring Company (Acquiring)                                                         2
Tagline (Acquiring)                                                                 [4]
Market Categories (Acquiring)                                      [39, 19, 16, 49, 31]
Year Founded (Acquiring)        

### Imputing the null values


In [225]:
def knn_impute_numeric(df: pd.DataFrame, n_neighbors: int = 5) -> pd.DataFrame:

    df_copy = df.copy()

    numeric_cols = df_copy.select_dtypes(include=[float, int]).columns
    numeric_df = df_copy[numeric_cols]

    imputer = KNNImputer(n_neighbors=n_neighbors)
    imputed_array = imputer.fit_transform(numeric_df)

    imputed_df = pd.DataFrame(imputed_array, columns=numeric_cols, index=df_copy.index)
    df_copy[numeric_cols] = imputed_df

    return df_copy

In [226]:
df.isnull().sum().sum()

np.int64(213)

In [227]:
df = knn_impute_numeric(df.infer_objects())

In [228]:
df.isnull().sum().sum()  # Tagline

np.int64(3)

In [229]:
numeric_df = df.select_dtypes(include=[float, int])
correlations = numeric_df.drop("Price", axis=1).apply(
    lambda x: abs(x.corr(numeric_df["Price"], method="kendall"))
)

In [230]:
correlations.sort_values(ascending=False)

Age on acquisition                    0.346020
Year Founded (Acquiring)              0.206454
Total Funding ($) (Acquiring)         0.201643
IPO (Acquiring)                       0.178588
Country (HQ)                          0.167743
Number of Employees (Acquiring)       0.152101
State / Region (HQ) (Acquiring)       0.094667
Country (HQ) (Acquiring)              0.079755
State / Region (HQ)                   0.049958
Acquiring Company (Acquiring)         0.048023
City (HQ) (Acquiring)                 0.030493
Status (Acquisitions)                 0.017956
Company                               0.014899
City (HQ)                             0.014496
Number of Acquisitions (Acquiring)    0.014488
dtype: float64

In [231]:
df.head()

,Company,Tagline,Market Categories,City (HQ),State / Region (HQ),Country (HQ),Price,Age on acquisition,Acquiring Company (Acquiring),Tagline (Acquiring),...,Number of Employees (Acquiring),Total Funding ($) (Acquiring),Number of Acquisitions (Acquiring),Board Members (Acquiring),City (HQ) (Acquiring),State / Region (HQ) (Acquiring),Country (HQ) (Acquiring),Acquired Companies (Acquiring),Status (Acquisitions),Terms (Acquisitions)
0,66.0,Day Software develops web applications that al...,[117],8.0,2.0,13.0,81.0,8.4,2.0,[4],...,8.0,0.0,14.0,[303],18.0,0.0,5.0,"[24, 56, 75, 103, 127, 154, 180, 354, 364, 372...",2.0,[0]
1,73.0,"Efficient Frontier, an online performance and ...",[1],101.0,5.0,15.0,106.0,10.0,2.0,[4],...,8.0,0.0,14.0,[303],18.0,0.0,5.0,"[24, 56, 75, 103, 127, 154, 180, 354, 364, 372...",2.0,[2]
2,138.0,Macromedia is a graphics and web development s...,[117],91.0,5.0,15.0,158.0,14.0,2.0,[4],...,8.0,0.0,14.0,[303],18.0,0.0,5.0,"[24, 56, 75, 103, 127, 154, 180, 354, 364, 372...",2.0,[1]
3,163.0,Neolane is a marketing technology provider dev...,[117],2.0,19.0,5.0,118.0,13.0,2.0,[4],...,8.0,0.0,14.0,[303],18.0,0.0,5.0,"[24, 56, 75, 103, 127, 154, 180, 354, 364, 372...",2.0,[2]
4,179.0,Omniture is an online marketing and web analyt...,"[117, 1]",73.0,44.0,15.0,146.0,14.0,2.0,[4],...,8.0,0.0,14.0,[303],18.0,0.0,5.0,"[24, 56, 75, 103, 127, 154, 180, 354, 364, 372...",2.0,[1]


In [232]:
df['Tagline'].isnull().sum()

np.int64(3)

In [233]:
df['Tagline'] = acquired['Tagline'].fillna('')


In [234]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

df['Tagline_Embedding'] = acquired['Tagline'].apply(lambda x: model.encode(str(x)).tolist())
df['Tagleline (aquiring)_Emb']=acquiring['Tagline (Acquiring)'].apply(lambda x: model.encode(str(x)).tolist())


In [235]:
df = df.drop("Tagline",axis=1)
df = df.drop("Tagline (Acquiring)",axis=1)

In [236]:
df.head()

,Company,Market Categories,City (HQ),State / Region (HQ),Country (HQ),Price,Age on acquisition,Acquiring Company (Acquiring),Market Categories (Acquiring),Year Founded (Acquiring),...,Number of Acquisitions (Acquiring),Board Members (Acquiring),City (HQ) (Acquiring),State / Region (HQ) (Acquiring),Country (HQ) (Acquiring),Acquired Companies (Acquiring),Status (Acquisitions),Terms (Acquisitions),Tagline_Embedding,Tagleline (aquiring)_Emb
0,66.0,[117],8.0,2.0,13.0,81.0,8.4,2.0,"[39, 19, 16, 49, 31]",12.0,...,14.0,[303],18.0,0.0,5.0,"[24, 56, 75, 103, 127, 154, 180, 354, 364, 372...",2.0,[0],"[-0.03807871416211128, -0.01274933572858572, 0...","[0.03632040694355965, -0.06904473155736923, -0..."
1,73.0,[1],101.0,5.0,15.0,106.0,10.0,2.0,"[39, 19, 16, 49, 31]",12.0,...,14.0,[303],18.0,0.0,5.0,"[24, 56, 75, 103, 127, 154, 180, 354, 364, 372...",2.0,[2],"[0.06601767241954803, -0.11639842391014099, -0...","[0.024867534637451172, -0.05182823911309242, -..."
2,138.0,[117],91.0,5.0,15.0,158.0,14.0,2.0,"[39, 19, 16, 49, 31]",12.0,...,14.0,[303],18.0,0.0,5.0,"[24, 56, 75, 103, 127, 154, 180, 354, 364, 372...",2.0,[1],"[0.05183260142803192, -0.06841101497411728, 0....","[-0.028975768014788628, -0.05876034498214722, ..."
3,163.0,[117],2.0,19.0,5.0,118.0,13.0,2.0,"[39, 19, 16, 49, 31]",12.0,...,14.0,[303],18.0,0.0,5.0,"[24, 56, 75, 103, 127, 154, 180, 354, 364, 372...",2.0,[2],"[0.037023697048425674, -0.0282935481518507, -0...","[0.0736999437212944, -0.03264852985739708, 0.0..."
4,179.0,"[117, 1]",73.0,44.0,15.0,146.0,14.0,2.0,"[39, 19, 16, 49, 31]",12.0,...,14.0,[303],18.0,0.0,5.0,"[24, 56, 75, 103, 127, 154, 180, 354, 364, 372...",2.0,[1],"[-0.009346939623355865, 0.08956177532672882, -...","[-0.07472138851881027, 0.008474593982100487, 0..."


# TODO
* scaling
* outliers
* check that common categories are given the same value in encoding
* for values encoded as arrays: remove the brackets for the one-only elements
* embed Tagline
* What to do with founders